In [1]:
import os
import sys

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

In [5]:
from tqdm import tqdm
from datasets import load_dataset
from src.decoder_utils import calculate_aggregated_hidden_states

%matplotlib inline
%config InlineBackend.figure_format='retina'

2025-09-23 19:06:21 DEBUG    matplotlib.pyplot: Loaded backend inline version unknown.


### Dataset loading

In [6]:
ds = load_dataset("ContributorsSIGNAL/SIGNAL")

2025-09-23 19:06:22 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /datasets/ContributorsSIGNAL/SIGNAL/resolve/main/README.md HTTP/1.1" 307 0
2025-09-23 19:06:23 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /api/resolve-cache/datasets/ContributorsSIGNAL/SIGNAL/a21068568606a9b51a27e23ea3edbc78b5b6478c/README.md HTTP/1.1" 200 0
2025-09-23 19:06:23 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /datasets/ContributorsSIGNAL/SIGNAL/resolve/a21068568606a9b51a27e23ea3edbc78b5b6478c/SIGNAL.py HTTP/1.1" 404 0
2025-09-23 19:06:23 DEBUG    urllib3.connectionpool: Resetting dropped connection: s3.amazonaws.com
2025-09-23 19:06:23 DEBUG    urllib3.connectionpool: https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/ContributorsSIGNAL/SIGNAL/ContributorsSIGNAL/SIGNAL.py HTTP/1.1" 404 0
2025-09-23 19:06:23 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /datasets/ContributorsSIGNAL/SIGNAL/resolve/a21

In [7]:
df = ds["train"].to_pandas()

In [8]:
congruent_sentences = (df["congruent"].tolist())
congruent_sentences_unique = (df["congruent"].unique().tolist())
incongruent_sentences = (df["sentence"].tolist())

In [9]:
len(congruent_sentences), len(incongruent_sentences)

(600, 600)

In [10]:
model_names = [
    "openai-community/gpt2"
]

In [11]:
model_outputs = dict()

for model_name in tqdm(model_names, desc="Infer incongruent sentences"):
    model_outputs[model_name] = calculate_aggregated_hidden_states(
        congruent_sentences=congruent_sentences,
        incongruent_sentences=incongruent_sentences,
        model_name=model_name,
        device="mps",
    )

Infer incongruent sentences:   0%|          | 0/1 [00:00<?, ?it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

2025-09-23 19:06:31 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /openai-community/gpt2/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
2025-09-23 19:06:31 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /api/resolve-cache/models/openai-community/gpt2/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer_config.json HTTP/1.1" 200 0
/opt/miniconda3/envs/SIGNAL-analysis/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted 

In [17]:
hidden_states_folder = "../persisted_hidden_states"
os.makedirs(hidden_states_folder, exist_ok=False)

FileExistsError: [Errno 17] File exists: '../persisted_hidden_states'

In [29]:
import numpy as np

for model_name, states in model_outputs.items():
    save_dir = hidden_states_folder + "/" + model_name.replace("/", "-")
    os.makedirs(save_dir, exist_ok=True)
    for i, sentence_result_dict in enumerate(states):
        aggregated_word_embedding_by_layer = sentence_result_dict["aggregated_word_embedding_by_layer"]
        np.save(save_dir + "/" + str(i) + ".npy", aggregated_word_embedding_by_layer.numpy())

In [25]:
model_outputs['openai-community/gpt2'][0].keys()

dict_keys(['text', 'hidden_states', 'offset_mapping', 'incongruent_word_start_end_indices', 'aggregated_word_embedding_by_layer'])